In [1]:
## which state has the most sightings? over time?
## most frequent shape by state?
## timeline changes? trends?
## sighting density around airports/afb

In [2]:
import os
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st

In [3]:
ufo_df = pd.read_csv('resources/ufo.csv', low_memory=False)
airport_df = pd.read_csv('resources/airports_clean.csv', low_memory=False)

In [4]:
nRow, nCol = ufo_df.shape
print(f'This csv has {nRow} rows and {nCol} columns')

This csv has 70938 rows and 13 columns


In [5]:
#split datetime into date and time and move them to the front of the df

ufo_df[['date', 'time']] = ufo_df.datetime.str.split(' ', expand=True)

first_column = ufo_df.pop('time')
ufo_df.insert(0, 'time', first_column)

first_column = ufo_df.pop('date')
ufo_df.insert(0, 'date', first_column)

ufo_df.pop('datetime')
       
ufo_df.head()

,date,time,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,region,sighting
0,5/8/2014,0:00,memphis,tn,us,rectangle,900,15 minutes,Standing at my window around 0:00 brilliantly ...,5/8/2014,35.149444,-90.048889,Southeast,1
1,5/7/2014,23:30,san isidro,nm,us,unknown,15,15 seconds,2 red lights gliding across sky&#44 then green...,5/8/2014,35.563363,-106.770591,Southwest,2
2,5/7/2014,21:20,hillsboro,mo,us,fireball,360,4-6 minutes,Round slow moving silent ball looked like a ca...,5/8/2014,38.232222,-90.562778,Midwest,3
3,5/7/2014,21:10,naugatuck,ct,us,unknown,300,5 minutes,Large formation of lights in western sky&#44 f...,5/8/2014,41.485833,-73.051111,Northeast,4
4,5/7/2014,21:00,theodore,al,us,circle,120,few minutes,Driving home and entered my neighborhood and s...,5/8/2014,30.547500,-88.175278,Southeast,5


In [6]:
#convert 'duration' column from object to float64

ufo_df.rename(columns = {'duration (seconds)':'duration_sec'}, inplace = True)

ufo_df["duration_sec"] = pd.to_numeric(ufo_df.duration_sec, errors='coerce')

In [7]:
ufo_trim = pd.read_csv('resources/ufo.csv', usecols = ['latitude','longitude','sighting'], low_memory=False)
ufo_trim["latitude"] = ufo_trim["latitude"].astype(np.float32)
ufo_trim["longitude"] = ufo_trim["longitude"].astype(np.float32)
ufo_trim["sighting"] = ufo_trim["sighting"].astype(np.int16)

ufo_trim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70938 entries, 0 to 70937
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   70938 non-null  float32
 1   longitude  70938 non-null  float32
 2   sighting   70938 non-null  int16  
dtypes: float32(2), int16(1)
memory usage: 692.9 KB


In [8]:
ufo_trim.head()

,latitude,longitude,sighting
0,35.149445,-90.048889,1
1,35.563362,-106.770592,2
2,38.232224,-90.562775,3
3,41.485832,-73.051109,4
4,30.547501,-88.175278,5


In [9]:
airport_trim = pd.read_csv('resources/airports.csv', usecols = ['name','latitude2','longitude2'], low_memory=False)
airport_trim["latitude2"] = airport_trim["latitude2"].astype(np.float32)
airport_trim["longitude2"] = airport_trim["longitude2"].astype(np.float32)
airport_trim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23910 entries, 0 to 23909
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        23910 non-null  object 
 1   latitude2   23910 non-null  float32
 2   longitude2  23910 non-null  float32
dtypes: float32(2), object(1)
memory usage: 373.7+ KB


In [10]:
airport_trim.head()

,name,latitude2,longitude2
0,Los Angeles International Airport,33.942501,-118.407997
1,Chicago O'Hare International Airport,41.978600,-87.904800
2,John F Kennedy International Airport,40.639801,-73.778900
3,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101
4,San Francisco International Airport,37.618999,-122.375000


In [ ]:
#idx = pd.MultiIndex.from_product([ufo_trim.index, airport_trim.index], names=['sight', 'airport'])

In [ ]:
#pair_df = pd.DataFrame(index=idx) \
#        .join(ufo_trim[['latitude', 'longitude']], on='sight') \
#        .join(airport_trim[['latitude2', 'longitude2']], on='airport')
#pair_df.info()

In [11]:
# Renaming the column names 
ufo_trim=ufo_trim.rename(columns = {'latitude':'lat','longitude':'lon'})
airport_trim=airport_trim.rename(columns = {'latitude2':'lat','longitude2':'lon'})
# To make sure that there are no null values and All are either integers/ Float values 
ufo_trim.info() 
print('\n XXXXXXXXXXXXXXXXXXXXXXX\n')
airport_trim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70938 entries, 0 to 70937
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   lat       70938 non-null  float32
 1   lon       70938 non-null  float32
 2   sighting  70938 non-null  int16  
dtypes: float32(2), int16(1)
memory usage: 692.9 KB

 XXXXXXXXXXXXXXXXXXXXXXX

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23910 entries, 0 to 23909
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    23910 non-null  object 
 1   lat     23910 non-null  float32
 2   lon     23910 non-null  float32
dtypes: float32(2), object(1)
memory usage: 373.7+ KB


In [12]:
from math import radians, cos, sin, asin, sqrt
def dist(lat1, long1, lat2, long2):
    """
Replicating the same formula as mentioned in Wiki
    """
    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km3

In [13]:
#pair_df['distance'] = haversine_np(*pair_df[ufo_df['latitude], ufo_df['longitude'], airport_df['latitude2'], airport_df['longitude2']].values.T)
#pair_df.info()

In [14]:
#pair_df.head()

In [15]:
# The closest airporty to each sighting
#min_distance = pair_df.loc[pair_df.groupby('sighting')['distance'].idxmin(), 'distance']

In [16]:
# Pulling everything together
#min_distance.to_frame().join(ufo_trim, on='sighting').join(airport_trim, on='name') \
#    [['sighting', 'name', 'distance']]

In [18]:
def find_nearest(lat, long):
    distances = airport_trim.apply(
        lambda row: dist(lat, long, row['lat'], row['lon']), 
        axis=1)
    return airport_trim.loc[distances.idxmin(), 'name']

In [19]:
ufo_trim['name'] = ufo_trim.apply(
    lambda row: find_nearest(row['lat'], row['lon']), 
    axis=1)
# To check the data frame if it has a new column of hotel name (for each and every member's location in the list)
ufo_trim.head()

,lat,lon,sighting,name
0,35.149445,-90.048889,1,67 Madison Ave Partnership Ltd Heliport
1,35.563362,-106.770592,2,Sandoval Regional Medical Center Heliport
2,38.232224,-90.562775,3,Christian Outreach Airfield
3,41.485832,-73.051109,4,Rondo Heliport
4,30.547501,-88.175278,5,Dale O. Galer Aerodrome


In [21]:
ufo_trim = pd.merge(ufo_trim,airport_trim[['name','lat','lon']],on='name', how='left')
# Rename the new columns as both the columns has same name, and python gets confused 

ufo_trim=ufo_trim.rename(columns = {'lat_x':'m_lat','lon_x':'m_lon','lat_y':'h_lat','lon_y':'h_lon'})

ufo_trim.head()

,m_lat,m_lon,sighting,name,h_lat,h_lon
0,35.149445,-90.048889,1,67 Madison Ave Partnership Ltd Heliport,35.137299,-90.054298
1,35.563362,-106.770592,2,Sandoval Regional Medical Center Heliport,35.308094,-106.680969
2,38.232224,-90.562775,3,Christian Outreach Airfield,38.304218,-90.527153
3,41.485832,-73.051109,4,Rondo Heliport,41.508400,-73.037300
4,30.547501,-88.175278,5,Dale O. Galer Aerodrome,30.491982,-88.197304


In [24]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

# Creating a new column to generate the output by passing lat long information to Haversine Equation
ufo_trim['distance'] = [haversine(ufo_trim.m_lon[i],ufo_trim.m_lat[i],ufo_trim.h_lon[i],ufo_trim.h_lat[i]) for i in range(len(ufo_trim))]
ufo_trim['distance'] = ufo_trim['distance'].round(decimals=3)
# Printing the data table 
ufo_trim.head(100)

,m_lat,m_lon,sighting,name,h_lat,h_lon,distance
0,35.149445,-90.048889,1,67 Madison Ave Partnership Ltd Heliport,35.137299,-90.054298,1.437
1,35.563362,-106.770592,2,Sandoval Regional Medical Center Heliport,35.308094,-106.680969,29.523
2,38.232224,-90.562775,3,Christian Outreach Airfield,38.304218,-90.527153,8.588
3,41.485832,-73.051109,4,Rondo Heliport,41.508400,-73.037300,2.760
4,30.547501,-88.175278,5,Dale O. Galer Aerodrome,30.491982,-88.197304,6.524
...,...,...,...,...,...,...,...
95,34.787777,-82.692497,73,Davis Field,42.616699,-92.216904,1198.836
96,34.787777,-82.692497,73,Davis Field,30.797501,-82.027603,448.026
97,34.787777,-82.692497,73,Davis Field,41.997829,-84.476471,816.601
98,34.787777,-82.692497,73,Davis Field,34.765301,-82.691101,2.502


In [43]:
distance_bins = [0, 5, 25.00001, 125.00001, 625.00001, 1000, 99999]
distance_ranges = ["<5", "5-25", "25-125", "125-625", "625-999", "1000+"]
sighting_total = ufo_trim['sighting'].count()

ufo_trim["Distance (km)"] = pd.cut(ufo_trim["distance"], distance_bins, labels=distance_ranges)
sighting_count = ufo_trim.groupby(["Distance (km)"]).count()["sighting"].rename("Sightings")
sighting_percent = sighting_count / sighting_total * 100
binned_distance = pd.DataFrame({"Sightings": sighting_count, "Sighting Percent": sighting_percent})

#binned_distance

binned_distance.sort_index().style.format({"Sightings": "{:,.0f}",
                          "Sighting Percent": "{:,.2f}%"})


,Sightings,Sighting Percent
Distance (km),,
<5,"54,425",65.33%
5-25,"16,241",19.49%
25-125,610,0.73%
125-625,"2,075",2.49%
625-999,"2,451",2.94%
1000+,"7,507",9.01%
